In [29]:
import csv
import requests
from bs4 import BeautifulSoup
import math
import os
import pandas as pd
from time import sleep

In [30]:
URL = 'https://zakup.nationalbank.kz/ru/publics/buys'
# для имитации работы браузера
HEADERS = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36', 'accept': '*/*'}
HOST = 'https://zakup.nationalbank.kz'
FILE = 'tenders.csv'

#get request
def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

# находим реквезит по индексу     
def get_requisites(link):
    html = get_html(link)
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, 'html.parser')
        rec = soup.find("table", {"id": "w0"}).find_all('td')[6].text.replace('\n',
                        ', ').replace('город','г.').replace('район',
                        'р.').replace('Проспект','п.').replace('здание','зд.').replace('Название банка','Банк')
        return rec
    else:
        print('Error')
#берем общее количество элементов и делим на кол. на одной странице и округляем 
def get_pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    perpage = soup.find('div', class_='summary').find_all('b')[0].text.replace('1-','')
    amount =  soup.find('div', class_='summary').find_all('b')[1].text.replace(u'\xa0','')
    pages=math.ceil(int(amount)/int(perpage))
    return pages

def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('tr', class_='item')
    tenders = []
    for item in items:
        td_data=item.find_all('td', class_="hidden-xs")
#элементы с одинаковыми классами записываем в лист
        values = [ele.text.strip() for ele in td_data]
        keys=['method','b_date','e_date','organizer','status']
#можно было просто по индексу взять но я записала в словарь
        td=dict(zip(keys, values))
        link=item.find('a', class_='word-break').get('href')
        rec=get_requisites(link)
        title = item.find('a', class_='word-break').get_text()
        b_date =td['b_date']
        e_date =td['e_date']
        status =td['status']
        requisites= rec
        dataset.loc[len(dataset)] = [title, b_date, e_date, status, requisites]
        

#         
def parse():
    html = get_html(URL)
    if html.status_code == 200:
#         pages=2
        pages=get_pages_count(html.text)
        for page in range(1, pages):
            print(f'Парсинг страницы {page} из {pages}...')
            html = get_html(URL, params={'page': page})
            get_content(html.text)
    else:
        print('Error')

In [31]:
dataset = pd.DataFrame(columns=["Название","Дата начала","Дата окончания","Статус организатора","Банковские реквизиты"])
parse()

Парсинг страницы 1 из 114...
Парсинг страницы 2 из 114...
Парсинг страницы 3 из 114...
Парсинг страницы 4 из 114...
Парсинг страницы 5 из 114...
Парсинг страницы 6 из 114...
Парсинг страницы 7 из 114...
Парсинг страницы 8 из 114...
Парсинг страницы 9 из 114...
Парсинг страницы 10 из 114...
Парсинг страницы 11 из 114...
Парсинг страницы 12 из 114...
Парсинг страницы 13 из 114...
Парсинг страницы 14 из 114...
Парсинг страницы 15 из 114...
Парсинг страницы 16 из 114...
Парсинг страницы 17 из 114...
Парсинг страницы 18 из 114...
Парсинг страницы 19 из 114...
Парсинг страницы 20 из 114...
Парсинг страницы 21 из 114...
Парсинг страницы 22 из 114...
Парсинг страницы 23 из 114...
Парсинг страницы 24 из 114...
Парсинг страницы 25 из 114...
Парсинг страницы 26 из 114...
Парсинг страницы 27 из 114...
Парсинг страницы 28 из 114...
Парсинг страницы 29 из 114...
Парсинг страницы 30 из 114...
Парсинг страницы 31 из 114...
Парсинг страницы 32 из 114...
Парсинг страницы 33 из 114...
Парсинг страницы 34

In [33]:
dataset

,Название,Дата начала,Дата окончания,Статус организатора,Банковские реквизиты
0,Питьевая вода (г. Нур-Султан),2021-10-29 17:00:00,2021-11-05 17:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ39125KZT1001305542, КБЕ:..."
1,Засходные материалы ГИТ,2021-11-01 09:00:00,2021-11-08 09:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ77125KZT1001300105, КБЕ:..."
2,Системный блок,2021-11-01 09:00:00,2021-11-08 09:00:00,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ706010131000043597, КБЕ:..."
3,Лак,2021-10-29 15:30:00,2021-11-08 09:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ77125KZT1001300105, КБЕ:..."
4,Ботинки,2021-10-29 15:00:00,2021-11-05 10:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ77125KZT1001300105, КБЕ:..."
5,Услуги обучения по вопросам DB-I Введение в ад...,2021-10-29 14:00:00,2021-10-29 15:39:49,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ706010131000043597, КБЕ:..."
6,Товарно-материальные запасы (рукавицы хлопчато...,2021-11-01 10:00:00,2021-11-08 10:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ39125KZT1001305542, КБЕ:..."
7,Гарнитуры (наушники с микрофоном),2021-10-29 12:00:00,2021-11-05 12:00:00,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ706010131000043597, КБЕ:..."
8,"Маска, средство дезинфицирующее, термометр по ...",2021-11-01 09:00:00,2021-11-08 09:00:00,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ776010131000118840, КБЕ:..."
9,Работы по текущему ремонту фасада здания,2021-11-01 10:00:00,2021-11-08 10:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ39125KZT1001305542, КБЕ:..."


In [34]:
dataset.to_csv('tenders.csv',encoding='utf-8-sig', sep=';',index = False)

In [35]:
data=pd.read_csv('tenders.csv',sep=';')
data

,Название,Дата начала,Дата окончания,Статус организатора,Банковские реквизиты
0,Питьевая вода (г. Нур-Султан),2021-10-29 17:00:00,2021-11-05 17:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ39125KZT1001305542, КБЕ:..."
1,Засходные материалы ГИТ,2021-11-01 09:00:00,2021-11-08 09:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ77125KZT1001300105, КБЕ:..."
2,Системный блок,2021-11-01 09:00:00,2021-11-08 09:00:00,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ706010131000043597, КБЕ:..."
3,Лак,2021-10-29 15:30:00,2021-11-08 09:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ77125KZT1001300105, КБЕ:..."
4,Ботинки,2021-10-29 15:00:00,2021-11-05 10:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ77125KZT1001300105, КБЕ:..."
5,Услуги обучения по вопросам DB-I Введение в ад...,2021-10-29 14:00:00,2021-10-29 15:39:49,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ706010131000043597, КБЕ:..."
6,Товарно-материальные запасы (рукавицы хлопчато...,2021-11-01 10:00:00,2021-11-08 10:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ39125KZT1001305542, КБЕ:..."
7,Гарнитуры (наушники с микрофоном),2021-10-29 12:00:00,2021-11-05 12:00:00,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ706010131000043597, КБЕ:..."
8,"Маска, средство дезинфицирующее, термометр по ...",2021-11-01 09:00:00,2021-11-08 09:00:00,Опубликовано,"БИК: HSBKKZKX, ИИК: KZ776010131000118840, КБЕ:..."
9,Работы по текущему ремонту фасада здания,2021-11-01 10:00:00,2021-11-08 10:00:00,Опубликовано,"БИК: NBRKKZKX, ИИК: KZ39125KZT1001305542, КБЕ:..."
